# MNIST Iteration 1 - Training a Fast.AI model

After I downloaded the kaggle dataset, I was a bit surprised that it did not contain any "real" images, i.e. image files. Instead, the images are stored in a `csv`.file.

To get a quick result and a baseline, I decided to switch to the MMIST-dataset provided by Fast.AI for training my first model.

In [1]:
from fastai.vision.all import *

## Downloading the Fast.AI MNIST Dataset

Fast.AI has a version of MNIST available:

In [2]:
path = untar_data(URLs.MNIST)

In [3]:
path

Path('/home/chrwittm/.fastai/data/mnist_png')

In [4]:
path.ls()

(#2) [Path('/home/chrwittm/.fastai/data/mnist_png/training'),Path('/home/chrwittm/.fastai/data/mnist_png/testing')]

As a result, we have a the training dat and the testing data in separate folders.

In [5]:
(path/'training').ls()

(#10) [Path('/home/chrwittm/.fastai/data/mnist_png/training/4'),Path('/home/chrwittm/.fastai/data/mnist_png/training/7'),Path('/home/chrwittm/.fastai/data/mnist_png/training/9'),Path('/home/chrwittm/.fastai/data/mnist_png/training/5'),Path('/home/chrwittm/.fastai/data/mnist_png/training/8'),Path('/home/chrwittm/.fastai/data/mnist_png/training/0'),Path('/home/chrwittm/.fastai/data/mnist_png/training/2'),Path('/home/chrwittm/.fastai/data/mnist_png/training/1'),Path('/home/chrwittm/.fastai/data/mnist_png/training/6'),Path('/home/chrwittm/.fastai/data/mnist_png/training/3')]

In the training folder, there are folders for the `png`-pictures of each number, which we can use for the lables.

In [6]:
path = (path/'training')

In [7]:
path

Path('/home/chrwittm/.fastai/data/mnist_png/training')

## Creating a quick model

With this dataset, let's create a quick model.

Introducing a new variable `do_learn` which needs to be set to `True` for the training to actually happen. - This is just for efficiency, and we can always use the `export.pkl` for making the predictions once the training has been completed once.

In [8]:
do_learn = False

In [9]:
if do_learn:
    mnist1 = DataBlock(
        blocks=(ImageBlock, CategoryBlock), 
        get_items=get_image_files, 
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        get_y=parent_label,
        #item_tfms=[Resize(192, method='squish')]
    )
    dls = mnist1.dataloaders(path, bs=32)
    dls.show_batch(max_n=6)

In [10]:
if do_learn:
    learn = vision_learner(dls, resnet18, metrics=error_rate)
    learn.fine_tune(3)

In [11]:
if do_learn:
    learn.export()

In [12]:
if do_learn == False:
    learn = load_learner('export.pkl')

Independent on how the variable `do_learn` was set, we always have a learner now.

## Benchmarking

Running the training on my local machine took about 40 minutes:

![CPU](benchmark-cpu.png)

Running on a free Paperspace GPU-server took about 5 minutes, so roughly an order of magnitude:

![GPU](benchmark-free-gpu.png)

## Making predictions

Let's use the `learner` to make some predictions 

In [13]:
learn = load_learner('export.pkl')

In [14]:
path

Path('/home/chrwittm/.fastai/data/mnist_png/training')

In [15]:
im = PILImage.create(path/'0/1.png')
im

In [16]:
learn.dls.vocab

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [17]:
learn.predict(im)

('0',
 TensorBase(0),
 TensorBase([9.9875e-01, 3.0521e-04, 1.0101e-04, 1.0142e-04, 8.4972e-05,
             1.4241e-05, 4.1763e-06, 1.7488e-04, 2.6674e-04, 2.0044e-04]))

In [18]:
def classify_image(img):
    pred,pred_idx,probs = learn.predict(img)
    return dict(zip(learn.dls.vocab, map(float, probs)))

In [19]:
classify_image(im)

{'0': 0.9987469911575317,
 '1': 0.0003052067186217755,
 '2': 0.00010101054795086384,
 '3': 0.00010141720849787816,
 '4': 8.497152157360688e-05,
 '5': 1.4241136341297533e-05,
 '6': 4.176294623903232e-06,
 '7': 0.00017488478624727577,
 '8': 0.0002667366643436253,
 '9': 0.00020043794938828796}

I could run a lot more predictions, but let's move on to the next task, i.e. working with the kaggle data to make a submission.